<a href="https://colab.research.google.com/github/srv-sh/BanglaNewspaperClassification/blob/main/NewspaperClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
# পারমিশন দেখে নিন
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d furcifer/bangla-newspaper-dataset

In [ ]:
!unzip bangla-newspaper-dataset.zip

In [ ]:
import json 
with open('data_v2/data_v2.json',encoding = 'utf-8') as f:
  datastore = json.load(f)
sentences = []
labels = []
for item in datastore:
  sentences.append(item['content'])
  labels.append(item['category'])

In [ ]:
len(sentences)

In [ ]:
#lebels 
set(labels)

In [ ]:
# Encoding the labels 
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
label = encoder.fit_transform(labels)

In [ ]:
training_size = int(len(sentences)*.88)

training_sentences = sentences[:training_size]
training_labels = label[:training_size]
test_sentences = sentences[training_size:]
test_labels = label[training_size:]

In [ ]:
print(len(training_sentences))
print(len(test_sentences))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#tokenize dataset
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(training_sentences)


In [ ]:
tokenizer.word_index

In [ ]:
vocab_size = len(tokenizer.word_index) +1

In [ ]:
vocab_size

In [ ]:
 maxin= [len(training_sentences[i]) for i in range(len(training_sentences))]
 max_length = max(maxin)
 max_length = 500

In [ ]:
#padding 

trunc_type = 'post'
padding_type = 'post'

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,max_length,padding=padding_type,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
teesting_padded = pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [ ]:
print(testing_sequences[0])

In [ ]:
#this conversion is for tensorflow
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(teesting_padded)
testing_labels = np.array(test_labels)

In [ ]:
embedding_dim = 12


In [ ]:
model =  tf.keras.Sequential([
                              tf.keras.layers.Embedding(input_dim=vocab_size, 
                              output_dim=embedding_dim, 
                              input_length=max_length),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                              tf.keras.layers.Dense(9, activation='softmax')
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


In [ ]:
num_epochs = 5
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), callbacks=[tensorboard_callback], verbose=2)